# Intro

This project aims to check consumers opinios of a determined product or subject in Twitter. \
This is importat to understand consumers opinions and redirect the brand in the best possible direction based on the data of clients feelings. \
This is done by twitter extraction with tweepy and doing sentiment analysis with textblob.

# Code

In [2]:
import re     # regex
import tweepy # access the twitter api via python
from tweepy import OAuthHandler
from textblob import TextBlob # sentiment analysis

In [ ]:
class TwitterClient(object):
    # a class to interact with twitter api 
    # and do sentiment analysis
    
    
    def __init__(self):
        # initialization method for the class
        # autenticate to the twitter api
        
        # keys and tokens:
        consumer_key = 'XXXXX'
        consumer_secret = 'XXXXX'
        access_token = 'XXXXX'
        access_token_secret = 'XXXXX'
        
        # try to autenticate
        try: 
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # create OAuthHandler object
            self.auth.set_access_token(access_token, access_token_secret)
            # define access token and secret
            self.api = tweepy.API(self.auth)
            # create tweepy API object and autenticate
        except:
            print('Error: Authentication Failed')
            
            
    def clean_tweet(self,tweet):
        # remove links, special characters, ... using regex statements
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([ˆ0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
    
    
    def get_tweet_sentiment(self, tweet):
        # classify sentiment of the tweet using textblob
        
        # create TextBlob object of the tweet
        analysis = TextBlob(self.clean_tweet(tweet))
        # define sentiment
        if analysis. sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
        
    def get_tweets(self, query, count = 10):
        # main function to fetch and parse tweets
        
        # empty list to store parsed tweets
        tweets = []
        
        try:
            # call twitter api
            fetched_tweets = self.api.search(q = query, count = count)
            
            # parsing tweets
            for tweet in fetched_tweets:
                
                # empty dict to store required params of a tweet
                parsed_tweet = {}
                
                # saving the text of a tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
                
                # append parsed tweet te the list
                if tweet.retweet_count > 0:
                    # append only once if the tweet has retweets
                    if parsed_tweet not in tweets:
                        tweet.append(parsed_tweet)
                else:
                    # doesnt have retweets
                    tweets.append(parsed_tweet)
                    
            # returns parsed tweets
            return tweets
        
        # if anything goes wrong
        except tweepy.TweepError as e:
            # print error (if any)
            print('Error : ' + str(e))
            
            
            
def main():
    # create object of TwitterClient Class
    api = TwitterClient()
    # function to get the tweets
    # the query to search goes here
    tweets = api.get_tweets(query = 'Nike', count = 15)
    
    # separating the tweets for sentiment
    pos_tweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    neg_tweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    neu_tweets = [tweet for tweet in tweets if tweet['sentiment'] == 'neutral']
    
    # print the results
    print('Positive tweets percentage: {}%'.format( 100*len(pos_tweets)/len(tweets) ))
    print('Negative tweets percentage: {}%'.format( 100*len(neg_tweets)/len(tweets) ))
    print('Neutral tweets percentage: {}%'.format( 100*len(neu_tweets)/len(tweets) ))
    
    # print some tweets of each sentiment
    print('\n\n Positive tweets:')
    for tweet in pos_tweets[:2]:
        print(tweet['text'])
    
    print('\n\n Negative tweets:')
    for tweet in neg_tweets[:2]:
        print(tweet['text'])
    
    print('\n\n Neutral tweets:')
    for tweet in neu_tweets[:2]:
        print(tweet['text'])
        
        
if __name__ == '__main__':
    main()
    
    
# ideas:
# Get tweets for mentions not querys, like the mention of a brand in the tweet.